# runs simulations

In [1]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from helperfcts import *
from heap import Heap
from randomgraph import RandomGraph
from averagevalues import AverageValues

In [6]:
no_agents = 10
H = Heap(no_agents)
mu = 0 # average value
sigma = 1 # gaussian noise
AvgVal = AverageValues(no_agents, mu, sigma)
graph_size = 20 # n
edge_probability = 0.5 # p
Gnp = RandomGraph(graph_size, edge_probability)